In [35]:
import matplotlib.pyplot as plt
import os
import numpy as np
import math
import subprocess

> **Note:** This cell below is optional.  
> The bash script often printed extra comments, which cluttered the output.  
> Running this cell simply helps keep the output cleaner.

In [36]:
import os

# Strip out the exported bash functions that break Jupyter subshells
for key in list(os.environ):
    if key.startswith("BASH_FUNC_"):
        del os.environ[key]

print("✅ Cleaned exported bash functions. Subshells will be quiet now.")


✅ Cleaned exported bash functions. Subshells will be quiet now.


> Sometimes you may want to run code in a different directory than where this `.ipynb` notebook is located.  
> The cell below sets the working directory for everything you run after executing it.

In [37]:
# set working directory for all future cells
os.chdir("/panfs/accrepfs.vampire/data/kojetin_lab/Mithun/Covalent_ligand_redone")
# and check with bash command pwd - note the exclamation mark at the beginning
!pwd
# or in python
os.getcwd()

/panfs/accrepfs.vampire/data/kojetin_lab/Mithun/Covalent_ligand_redone


'/panfs/accrepfs.vampire/data/kojetin_lab/Mithun/Covalent_ligand_redone'

> **Important:**  
> These scripts are tailored for **ACCRE** and load modules available on ACCRE at Vanderbilt University.
> 
> **Option 1 – Adjust for your HPC:**  
> Edit the `module load` lines to match the Amber modules you would like to use
> **Option 2 – Source Amber.sh:**  
> If your cluster doesn’t use modules, remove those lines and source the AMBER environment , e.g.:
> ```bash
> source /sb/apps/amber22/amber.sh
> ```
> This ensures the correct AMBER version is used on any cluster.


In [38]:
import os, subprocess

def load_amber_env():
    """Load AmberTools + Amber environment into the Jupyter kernel."""
    bash_cmd = """
    module purge
    module load StdEnvACCRE/2023
    module load gcc/12.3 openmpi/4.1.5 cuda/12.2
    module load ambertools/25.0
    module load scipy-stack/2025a
    module load amber/24
    env
    """
    env_output = subprocess.check_output(["bash", "-c", bash_cmd], text=True)
    for line in env_output.splitlines():
        if "=" in line:
            k, v = line.split("=", 1)
            os.environ[k] = v
    print("✅ Amber environment loaded into Jupyter kernel.")


> Run the next cell to load the **AmberTools + Amber environment** into your Jupyter kernel.  
> This ensures all subsequent commands use the correct modules and environment variables.


In [ ]:
load_amber_env()

In [40]:
!mkdir Minimization
!cp ligand_prep/frcmod2.t007  Minimization
!cp ligand_prep/frcmod1.t007  Minimization
!cp ligand_preps/t007.prepin  Minimization

%cd Minimization 
#Couldnt copy t007.prepin for some reason so manually copied it 

cp: cannot stat 'ligand_preps/t007.prepin': No such file or directory
/panfs/accrepfs.vampire/data/kojetin_lab/Mithun/Covalent_ligand_redone/Minimization


/cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/Compiler/gcccore/ipykernel/2025a/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Writing the tleap script to Production folder so that we can provide relative paths for each file

In [ ]:
%%bash
cat > tleapd.sh <<'EOF'
#!/bin/bash
# load script using: tleap -s -f script-name.tleap

source leaprc.protein.ff14SB
source leaprc.gaff2
source leaprc.water.tip3p
loadamberparams frcmod.ionsjc_tip3p

loadAmberPrep t007.prepin
loadAmberParams ../ligand_prep/frcmod2.t007
loadAmberParams ../ligand_prep/frcmod1.t007

complex = loadpdb ../Models/PPARg_T007_combined1.pdb   # residue 285 renamed to T00; names match prepin

check complex
charge complex

addions complex Na+ 0
addions complex Cl- 0

solvateoct complex TIP3PBOX 10.0 iso

#addions complex K+ 16
#addions complex Cl- 16

saveamberparm complex prepin.prmtop prepin.inpcrd
savepdb       complex prepin.pdb
quit
EOF

echo "✅ tleapd.sh created"


✅ tleapd.sh created


In [42]:
!ls
load_amber_env()
!tleap -s -f tleapd.sh

frcmod1.t007  frcmod2.t007  t007.prepin  tleapd.sh


The following modules were not unloaded:
  (Use "module --force purge" to unload all):

  1) CCconfig        6)  ucx/1.14.1         11) flexiblas/3.3.1
  2) gentoo/2023     7)  libfabric/1.18.0   12) imkl/2023.2.0
  3) gcccore/.12.3   8)  pmix/4.2.4         13) StdEnvACCRE/2023
  4) gcc/12.3        9)  ucc/1.2.0
  5) hwloc/2.9.1     10) openmpi/4.1.5
Lmod Warning:
-------------------------------------------------------------------------------
The following dependent module(s) are not currently loaded: scipy-stack/2023b
(required by: plumed/2.9.0, ambertools/25.0)
-------------------------------------------------------------------------------




The following have been reloaded with a version change:
  1) ipykernel/2023b => ipykernel/2025a
  2) scipy-stack/2023b => scipy-stack/2025a



✅ Amber environment loaded into Jupyter kernel.
/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/sh: module: line 1: syntax error: unexpected end of file
/bin/sh: error importing function definition for `module'
/bin/sh: ml: line 1: syntax error: unexpected end of file
/bin/sh: error importing function definition for `ml'
-I: Adding /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/ambertools/25.0/dat/leap/prep to search path.
-I: Adding /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/ambertools/25.0/dat/leap/lib to search path.
-I: Adding /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/ambertools/25.0/dat/leap/parm to search path.
-I: Adding /cvmfs/soft.computecanada.ca/easybuild/software/2023/

Lmod Warning:
-------------------------------------------------------------------------------
The following dependent module(s) are not currently loaded: scipy-stack/2025a
(required by: amber/24)
-------------------------------------------------------------------------------




Inactive Modules:
  1) arrayfire/3.9.0     2) cudnn/8.9.5.29     3) glfw/3.3.8     4) spdlog/1.11.0

Due to MODULEPATH changes, the following have been reloaded:
  1) ambertools/25.0     2) plumed/2.9.0     3) ucc-cuda/1.2.0     4) ucx-cuda/1.14.1

The following have been reloaded with a version change:
  1) cuda/12.2 => cuda/12.6
  2) cudacore/.12.2.2 => cudacore/.12.6.2
  3) ipykernel/2025a => ipykernel/2023b
  4) nccl/2.18.3 => nccl/2.26.2
  5) scipy-stack/2025a => scipy-stack/2023b



Loading parameters: /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/ambertools/25.0/dat/leap/parm/gaff2.dat
Reading title:
AMBER General Force Field for organic molecules (Version 2.2.20, March 2021)
----- Source: /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/ambertools/25.0/dat/leap/cmd/leaprc.water.tip3p
----- Source of /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/ambertools/25.0/dat/leap/cmd/leaprc.water.tip3p done
Loading library: /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/ambertools/25.0/dat/leap/lib/atomic_ions.lib
Loading library: /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/ambertools/25.0/dat/leap/lib/solvents.lib
Loading parameters: /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/ambertools/25.0/dat/leap/parm/frcmod.tip3p
Reading force field modification type fil

### Run Minimization & Equilibration

This creates a `Minimization` directory, moves into it, and runs `mesmy.sh`.  

The `mesmy.sh` script performs a full **system relaxation protocol** using `pmemd.cuda`:

- **Step 1–5: Energy Minimization**  
  - Strongly restrains heavy atoms, then gradually relaxes the restraints until unrestrained.  
  - Uses steepest descent minimization to relieve steric clashes and relax solvent.

- **Step 6–9: Equilibration**  
  - Runs short NVT (constant volume) and NPT (constant pressure) MD simulations.  
  - Gradually reduces restraints on the protein and backbone atoms.  
  - Slowly increases timestep and stabilizes density and temperature.  

Each step writes:
- Restart files (`.rst7`) – used as input for the next step  
- Trajectories (`.nc`) – for visualization and analysis  
- Output logs (`.out`, `mdinfo`) – for monitoring progress  

By the end, the system is minimized, equilibrated, and ready for production MD.


In [43]:
load_amber_env()
!sh ../Scripts/mesmy.sh > log.mesmy

bash: ml: line 1: syntax error: unexpected end of file
bash: error importing function definition for `ml'
bash: module: line 1: syntax error: unexpected end of file
bash: error importing function definition for `module'
sh: module: line 1: syntax error: unexpected end of file
sh: error importing function definition for `module'
sh: ml: line 1: syntax error: unexpected end of file
sh: error importing function definition for `ml'
sh: module: line 1: syntax error: unexpected end of file
sh: error importing function definition for `module'
sh: ml: line 1: syntax error: unexpected end of file
sh: error importing function definition for `ml'
sh: module: line 1: syntax error: unexpected end of file
sh: error importing function definition for `module'
sh: ml: line 1: syntax error: unexpected end of file
sh: error importing function definition for `ml'
sh: module: line 1: syntax error: unexpected end of file
sh: error importing function definition for `module'
sh: ml: line 1: syntax error: unex

✅ Amber environment loaded into Jupyter kernel.
/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
sh: module: line 1: syntax error: unexpected end of file
sh: error importing function definition for `module'
sh: ml: line 1: syntax error: unexpected end of file
sh: error importing function definition for `ml'
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Note: The following 

### Hydrogen Mass Repartitioning (HMR)

This cell creates a small ParmEd input file (`hmr_commands.in`) that performs **Hydrogen Mass Repartitioning**  
and writes out a new topology file `prepinHMR.prmtop`.  
HMR redistributes a small amount of mass from heavy atoms to hydrogens,  
reducing calculations and making the simulation faster

In [44]:
%%bash
# Create a ParmEd input file with HMR commands
cat > hmr_commands.in <<EOF
HMassRepartition
outparm prepinHMR.prmtop
go
quit
EOF

# Run ParmEd with the input file
parmed prepin.prmtop < hmr_commands.in

bash: ml: line 1: syntax error: unexpected end of file
bash: error importing function definition for `ml'
bash: module: line 1: syntax error: unexpected end of file
bash: error importing function definition for `module'



       _________________________________________
   .'`                                           `'.
   |   _________________________________________   |
   | '\.---------------------------------------./' |
   | ||                                         || |
   | || PPPPPPP                                 || |
   | || P       P                               || |
   | || PPPPPPP                                 || |
   | || P                                       || |
   | || P   aaa  rrrrr.    m   m   eee  ddd     || |
   | || P  a   a r    r  m   m   m e    d   d   || |
   | || P  aaaaa r-rr.   m   m   m eee  d    d  || |
   | || P  a   a r    r  m   m   m e    d   d   || |
   | || P  a   a r     r m   m   m eee  ddd     || |
   | ||                                         || |
   | '/\_______________________________________/\' |
   '  `-----------------------------------------'  '
    \                                     .-.     /
     '.__~~~ ____()__()_()()() __________((_))__.'

In [46]:
%cd ../Production 
!ls

/panfs/accrepfs.vampire/data/kojetin_lab/Mithun/Covalent_ligand_redone/Production
/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
AMBER_err_5131392.log  gpu_production.slurm  production.pl  rep3
AMBER_out_5131392.log  prod1.in		     rep1
gpu_production1.slurm  prod1.in.start	     rep2


/cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/Compiler/gcccore/ipykernel/2025a/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


### Production MD Setup & Execution

Production MD is handled by a **Perl driver script** (`production.pl`) that automates running multiple  
**100 ns blocks** of MD back-to-back using `pmemd.cuda`. This allows you to build very long trajectories  
(e.g., microseconds) without manually restarting each run.

What `production.pl` does:
- Reads the final minimized/equilibrated coordinates (`step9.rst7`) and topology (`prepinHMR.prmtop`).
- Loops from `$nstart` to `$nend`, generating sequential 100 ns MD chunks:
  - Uses `prod1.in.start` for the first run (fresh start) and `prod1.in` for restarts.
  - Automatically names output files (`PTR_001.out`, `md_prod1_001.nc`, `md_prod1_001.rst`, etc.).
  - Chains restart files so each chunk continues from where the previous one ended.
- Prints and executes the `pmemd.cuda` command for each run.

Example:
- `$nend = 10` → runs 10 × 100 ns = **1 μs total production MD**.
- Increase `$nend` to extend the trajectory (e.g., 20 → 2 μs total).

The input files (`prod1.in.start` and `prod1.in`) specify:
- **Explicit solvent MD** under NPT conditions  
- **dt = 0.004 ps** (with SHAKE and HMR applied → 4 fs timestep)  
- Langevin thermostat (`ntt = 3`) and Monte Carlo barostat (`barostat = 2`)  
- Coordinates written every 10 ps (`ntwx = 2500`)  

> **Tip:**  
> Edit `production.pl` to adjust the number of chunks, file paths, or output naming as needed  
> before starting long production runs.


In [ ]:
#Test to see if it works 
!perl production.pl > PRODUCTION.OUT

BONUS - run 3 replicates at a time 

In [ ]:
%%bash
# === Setup replicate directories and copy required files ===
for rep in rep1 rep2 rep3; do
    if [ ! -d "$rep" ]; then
        echo "📂 Creating $rep"
        mkdir "$rep"
    fi

    # Copy production files if not already present
    for f in production.pl prod1.in prod1.in.start; do
        if [ ! -f "$rep/$f" ]; then
            echo "📄 Copying $f to $rep"
            cp "$f" "$rep/"
        fi
    done
done

# === Launch production runs in parallel ===
for rep in rep1 rep2 rep3; do
    echo "🔹 Starting production for $rep"
    cd "$rep"
    perl production.pl > "PRODUCTION_${rep}.OUT" 2> "ERROR_${rep}.LOG" &
    cd ..
done

# Wait for all background jobs to finish
wait
echo "✅ All replicates completed."
